# Dataframes merge

After exploring the data and doing minor cleaning we are going to merge the data of preinscriptions, matricules and notes so we end up with a unique dataframe that contains only those students that got matriculated after the preinscription and their grades.

In [28]:
import pandas as pd
import re
from unidecode import unidecode

In [29]:
preinscripcions_df = pd.read_csv('../data/processed/preinscripcions_df.csv')

matricules_df = pd.read_csv('../data/processed/matricules_df.csv')
notes_df = pd.read_csv('../data/processed/notes_df.csv')
titulacions_df = pd.read_csv('../data/processed/titulacions_df.csv')

In [30]:
print(f'Preinscripcions shape:{preinscripcions_df.shape}')
print(f'Matricules shape:{matricules_df.shape}')
print(f'Notes shape:{notes_df.shape}')

Preinscripcions shape:(2266, 33)
Matricules shape:(1713, 6)
Notes shape:(31963, 14)


### Merge of preinscripcions & matriculations

In [31]:
#We first merge preinscripcions and matricules to find out how many students do actually matriculate after preinscripcions.
#We try the join on DNI and codi_grau in case a student did preinscriptions and matriculated multiple times

In [32]:
check_data_merge = pd.merge(preinscripcions_df, matricules_df, on=['DNI'], how='outer', indicator=True)

In [33]:
check_data_merge.shape

(2422, 39)

In [34]:
check_data_merge.columns

Index(['curs_preinscripcio', 'convocatoria', 'DNI', 'codi_centre_secundaria',
       'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
       'sexe', 'data_naixement', 'via_acces', 'nota_acces', 'codi_grau_x',
       'ordre_assignacio', 'any_qualificacio', 'convocatoria_qualificacio',
       'estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'treball_remunerat', 'orientacio', 'aspectes_eleccio',
       'any_acabament_secundaria', 'tipus_centre_secundaria',
       'tipus_lloc_secundaria', 'codi_lloc_secundaria',
       'estudis_universitaris', 'any_acces_primera_vegada_universitat',
       'titulacio_nivell_assolit', 'modalitat_acces', 'identification',
       'curs_matricula', 'NIA', 'pla', 'codi_grau_y', '_merge'],
      dtype='object')

In [35]:
only_in_matricules = check_data_merge[check_data_merge['_merge'] == 'right_only'].copy()
print(only_in_matricules['curs_matricula'].value_counts())
only_in_matricules.shape
# 125 Rows only in matricules_df that do not appear in preinscriptions--> This shoudn't happen. 
#However, there is two casuistics that might explain this cases:
    # 1. Something wrong with 2019 data --> 70% of cases are from 2019
    # 2. Students used their passport for the preinscription and their DNI for matriculation. So there is no possible match.
    
    #only_in_matricules.to_excel('../data/only_in_matricules.xlsx', index=False)

curs_matricula
2019.0    84
2014.0    16
2016.0     9
2021.0     7
2015.0     5
2017.0     4
2022.0     3
2023.0     3
2018.0     2
2020.0     1
Name: count, dtype: int64


(134, 39)

In [36]:
first_merge = pd.merge(preinscripcions_df, matricules_df, on=['DNI','codi_grau'], how='inner')

In [37]:
first_merge.shape #1569 students ended up matriculating after the preinscription

(1560, 37)

In [38]:
#To have some context on numbers let's check how many rows appear in the preinscripcio list and in the matriculations list.
preinscripcions_counts = preinscripcions_df['curs_preinscripcio'].value_counts().sort_index()
matricula_counts = matricules_df['curs_matricula'].value_counts().sort_index()

counts_df = pd.DataFrame({
    'Curs_Preinscripcio': preinscripcions_counts,
    'Curs_Matricula': matricula_counts,
    'Count_Difference': preinscripcions_counts - matricula_counts
})

counts_df

,Curs_Preinscripcio,Curs_Matricula,Count_Difference
2014,203,180,23
2015,181,143,38
2016,202,173,29
2017,195,160,35
2018,178,124,54
2019,221,159,62
2020,191,133,58
2021,265,205,60
2022,318,217,101
2023,312,219,93


### Merge with grades

In [39]:
#The data in "notes_df" does not have grades for students matriculated on 2023. 
#We will use this students to test our model in the future. We save the students_2023 df.
#We divide this data to work with the rest of the data on our model. 

students_2023 = first_merge[first_merge['curs_matricula']== 2023]
students_2023.to_csv('../data/processed/students_2023.csv', index=False)

In [40]:
#Drop the rows from the students of 2023 from the preinscriptions and matriculations dataframe
first_merge = first_merge[first_merge['curs_matricula']!= 2023]

In [41]:
first_merge.shape

(1347, 37)

In [42]:
first_merge.columns

Index(['curs_preinscripcio', 'convocatoria', 'DNI', 'codi_centre_secundaria',
       'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
       'sexe', 'data_naixement', 'via_acces', 'nota_acces', 'codi_grau',
       'ordre_assignacio', 'any_qualificacio', 'convocatoria_qualificacio',
       'estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'treball_remunerat', 'orientacio', 'aspectes_eleccio',
       'any_acabament_secundaria', 'tipus_centre_secundaria',
       'tipus_lloc_secundaria', 'codi_lloc_secundaria',
       'estudis_universitaris', 'any_acces_primera_vegada_universitat',
       'titulacio_nivell_assolit', 'modalitat_acces', 'identification',
       'curs_matricula', 'NIA', 'pla'],
      dtype='object')

In [43]:
notes_df.columns

Index(['any_academic', 'assignatura', 'codi_assignatura', 'grup', 'NIA', 'pla',
       'tipus', 'curs', 'semestre', 'exhaurides', 'superada', 'nota_numerica',
       'nota_alfa', 'codi_grau'],
      dtype='object')

In [44]:
final_merge = pd.merge(first_merge,notes_df, on=['NIA','codi_grau','pla'], how='inner')

In [45]:
final_merge.columns

Index(['curs_preinscripcio', 'convocatoria', 'DNI', 'codi_centre_secundaria',
       'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
       'sexe', 'data_naixement', 'via_acces', 'nota_acces', 'codi_grau',
       'ordre_assignacio', 'any_qualificacio', 'convocatoria_qualificacio',
       'estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'treball_remunerat', 'orientacio', 'aspectes_eleccio',
       'any_acabament_secundaria', 'tipus_centre_secundaria',
       'tipus_lloc_secundaria', 'codi_lloc_secundaria',
       'estudis_universitaris', 'any_acces_primera_vegada_universitat',
       'titulacio_nivell_assolit', 'modalitat_acces', 'identification',
       'curs_matricula', 'NIA', 'pla', 'any_academic', 'assignatura',
       'codi_assignatura', 'grup', 'tipus', 'curs', 'semestre', 'exhaurides',
       'superada', 'nota_numerica', 'nota_alfa'],
      dtype='object')

In [46]:
#We want to keep only the rows of the 1st year of each student --> year where curs_matricula == any_academic (from notes)
condition = final_merge['curs_matricula'] == final_merge['any_academic']
final_merge = final_merge[condition]

In [47]:
final_merge = final_merge.drop(columns='identification')
final_merge = final_merge.drop(columns='DNI') #only need one identification column, redundant with NIA

In [48]:
final_merge.shape

(11378, 46)

In [49]:
final_merge.columns

Index(['curs_preinscripcio', 'convocatoria', 'codi_centre_secundaria',
       'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
       'sexe', 'data_naixement', 'via_acces', 'nota_acces', 'codi_grau',
       'ordre_assignacio', 'any_qualificacio', 'convocatoria_qualificacio',
       'estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'treball_remunerat', 'orientacio', 'aspectes_eleccio',
       'any_acabament_secundaria', 'tipus_centre_secundaria',
       'tipus_lloc_secundaria', 'codi_lloc_secundaria',
       'estudis_universitaris', 'any_acces_primera_vegada_universitat',
       'titulacio_nivell_assolit', 'modalitat_acces', 'curs_matricula', 'NIA',
       'pla', 'any_academic', 'assignatura', 'codi_assignatura', 'grup',
       'tipus', 'curs', 'semestre', 'exhaurides', 'superada', 'nota_numerica',
       'nota_alfa'],
      dtype='object')

In [50]:
order_columns = ['NIA','curs_preinscripcio', 'convocatoria','curs_matricula',   
                 'sexe','data_naixement','nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
                 'via_acces', 'nota_acces', 'ordre_assignacio','orientacio', 'aspectes_eleccio',
                 'any_qualificacio', 'convocatoria_qualificacio',
                 'treball_remunerat','estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
                'codi_centre_secundaria', 'any_acabament_secundaria', 'tipus_centre_secundaria','tipus_lloc_secundaria', 'codi_lloc_secundaria',
                'estudis_universitaris', 'any_acces_primera_vegada_universitat','titulacio_nivell_assolit', 'modalitat_acces',
                 'codi_grau', 'pla', 'any_academic', 'assignatura', 
                   'codi_assignatura', 'grup', 'tipus','curs','semestre', 'exhaurides',
                   'superada', 'nota_numerica', 'nota_alfa']

final_merge = final_merge[order_columns]

In [51]:
final_merge.to_csv('../data/raw/final_merge.csv', index=False)

In [52]:
final_merge.isnull().sum()

NIA                                        0
curs_preinscripcio                         0
convocatoria                               0
curs_matricula                             0
sexe                                       0
data_naixement                             0
nacionalitat                               0
pais                                       0
CP                                         0
poblacio                                   0
comarca                                  570
provincia                                 79
via_acces                                  0
nota_acces                                 0
ordre_assignacio                           0
orientacio                                 0
aspectes_eleccio                           0
any_qualificacio                           0
convocatoria_qualificacio                  0
treball_remunerat                          0
estudis_pare                               0
estudis_mare                               0
ocupacio_p